In [1]:
from openai import OpenAI
from dotenv import load_dotenv,find_dotenv
import json
import os
import requests

_:bool = load_dotenv(find_dotenv())
client :OpenAI= OpenAI()

In [2]:
key = os.environ["FMP_API_KEY"]


def income_statement(period):
    "Get income statement quarter or annual as per parameter or desire query"
    url = f"https://financialmodelingprep.com/api/v3/income-statement/AAPL?period={period}&apikey={key}"
    res = requests.get(url)
    return json.dumps(res.json())



def balance_sheet_statement(period):
    "Get balance sheet statement as per parameter"
    url = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/AAPL?period={period}&apikey={key}"
    res = requests.get(url)
    return json.dumps(res.json())


In [3]:
import json

def show_json(message, obj):
    display(message, json.loads(obj.model_dump_json()))

In [4]:
## ### Step 1: Create an Assistant and register/report your functions
from openai import OpenAI
_:bool = load_dotenv(find_dotenv())
client : OpenAI = OpenAI()
from openai.types.beta import Assistant

assistant: Assistant = client.beta.assistants.create(
  instructions="financial analytic specialist",
  model="gpt-3.5-turbo-1106",
  tools=[
      {
    "type": "function",
    "function": {
      "name": "income_statement",
      "description": "Get income statement quarter or annual as per parameter or desire query",
      "parameters": {
        "type": "object",
        "properties": {
          "period": {
            "type": "string", 
              "description": "income statement quarter or annual"}
        },
        "required": ["period"]
      }
    }
    
  },{
    "type": "function",
    "function": {
      "name": "balance_sheet_statement",
      "description": "Get balance sheet statement as per parameter",
      "parameters": {
        "type": "object",
        "properties": {
          "period": {
            "type": "string", 
              "description": "balance sheet statement"}
          },
        "required": ["period"]
        }
      }
    }]
)
dict(assistant)

{'id': 'asst_GF7yTBW5Epnl4YsozOVSwtxS',
 'created_at': 1702621901,
 'description': None,
 'file_ids': [],
 'instructions': 'financial analytic specialist',
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'name': None,
 'object': 'assistant',
 'tools': [ToolFunction(function=FunctionDefinition(name='income_statement', parameters={'type': 'object', 'properties': {'period': {'type': 'string', 'description': 'income statement quarter or annual'}}, 'required': ['period']}, description='Get income statement quarter or annual as per parameter or desire query'), type='function'),
  ToolFunction(function=FunctionDefinition(name='balance_sheet_statement', parameters={'type': 'object', 'properties': {'period': {'type': 'string', 'description': 'balance sheet statement'}}, 'required': ['period']}, description='Get balance sheet statement as per parameter'), type='function')]}

In [5]:
## Create a thread 

from openai.types.beta.thread import Thread

thread: Thread  = client.beta.threads.create()

dict(thread)


{'id': 'thread_zKIZJWwMm8U4dAtCm7TpqVzw',
 'created_at': 1702621916,
 'metadata': {},
 'object': 'thread'}

In [6]:
## ### Step 3: Add a Message to a Thread

from openai.types.beta.threads.thread_message import ThreadMessage

# First Request
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Kindly show income statement and balance sheet sheet statement"
)
dict(message)

{'id': 'msg_0lYmR8SyJbqxwhkAXDiAhlLh',
 'assistant_id': None,
 'content': [MessageContentText(text=Text(annotations=[], value='Kindly show income statement and balance sheet sheet statement'), type='text')],
 'created_at': 1702621923,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_zKIZJWwMm8U4dAtCm7TpqVzw'}

In [7]:
## Run the Assistant


from openai.types.beta.threads.run import Run

run: Run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)
dict(run)

{'id': 'run_WwxxMo9DBfZUcfHcEXd3suO2',
 'assistant_id': 'asst_GF7yTBW5Epnl4YsozOVSwtxS',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1702621929,
 'expires_at': 1702622529,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'financial analytic specialist',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_zKIZJWwMm8U4dAtCm7TpqVzw',
 'tools': [ToolAssistantToolsFunction(function=FunctionDefinition(name='income_statement', parameters={'type': 'object', 'properties': {'period': {'type': 'string', 'description': 'income statement quarter or annual'}}, 'required': ['period']}, description='Get income statement quarter or annual as per parameter or desire query'), type='function'),
  ToolAssistantToolsFunction(function=FunctionDefinition(name='balance_sheet_statement', parameters={'type': 'object', 'properties': {'period': {'type': 'string'

In [8]:
available_functions = {
    "income_statement": income_statement,
    "balance_sheet_statement": balance_sheet_statement
    
} 
dict(available_functions)

{'income_statement': <function __main__.income_statement(period)>,
 'balance_sheet_statement': <function __main__.balance_sheet_statement(period)>}

In [9]:
import time

# Loop until the run completes or requires action
while True:
    runStatus = client.beta.threads.runs.retrieve(thread_id=thread.id,
                                                  run_id=run.id)
    # Add run steps retrieval here for debuging
    run_steps = client.beta.threads.runs.steps.list(
        thread_id=thread.id, run_id=run.id)
    show_json("Run Steps:", run_steps)
    # print(runStatus.status ,'.....')

    # This means run is making a function call
    if runStatus.status == "requires_action":
        # print(runStatus.status ,'.....')
        # print("Status: ", "requires_action")
        show_json("submit_tool_outputs", runStatus.required_action)
        if runStatus.required_action.submit_tool_outputs and runStatus.required_action.submit_tool_outputs.tool_calls:

            toolCalls = runStatus.required_action.submit_tool_outputs.tool_calls

            tool_outputs = []
            for toolcall in toolCalls:
                function_name = toolcall.function.name
                function_args = json.loads(toolcall.function.arguments)

                if function_name in available_functions:

                    function_to_call = available_functions[function_name]
                    response = function_to_call(**function_args)
            tool_outputs.append({
                "tool_call_id": toolcall.id,
                "output": response
                })

                # print(tool_outputs, ">>>>>")
        # Submit tool outputs and update the run
            client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs)

    elif runStatus.status == "completed":
        # List the messages to get the response
        print("completed...........logic")
        messages: list[ThreadMessage] = client.beta.threads.messages.list(
            thread_id=thread.id)
        for message in messages.data:
            role_label = "User" if message.role == "user" else "Assistant"
            message_content = message.content[0].text.value
            print(f"{role_label}: {message_content}\n")
        break  # Exit the loop after processing the completed run

    elif run.status == "failed":
        print("Run failed.")
        break

    elif run.status in ["in_progress", "queued"]:
        print(f"Run is {run.status}. Waiting...")
        time.sleep(2)  # Wait for 5 seconds before checking again

    else:
        print(f"Unexpected status: {run.status}")
        break

'Run Steps:'

{'data': [],
 'object': 'list',
 'first_id': None,
 'last_id': None,
 'has_more': False}

Run is queued. Waiting...


'Run Steps:'

{'data': [],
 'object': 'list',
 'first_id': None,
 'last_id': None,
 'has_more': False}

...

{'data': [],
 'object': 'list',
 'first_id': None,
 'last_id': None,
 'has_more': False}

Run is queued. Waiting...


'Run Steps:'

{'data': [],
 'object': 'list',
 'first_id': None,
 'last_id': None,
 'has_more': False}

Run is queued. Waiting...
